1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

### взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Выбранный датасет: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

Prediction task is to determine whether a person makes over 50K a year.



In [1]:
import numpy as np
import pandas as pd

In [321]:
data = pd.read_csv('adult.data', index_col=False, names= ['age','workclass', 
                   'fnlwgt','education','education-num',
                   'marital-status','occupation', 'relationship',
                   'race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']) 

In [285]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [192]:
data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### сделать feature engineering

Целевая переменная - income. Требуется определить зарабатывает ли человек больше 50k в год.

In [322]:
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

In [323]:
data.rename(columns={'native-country': 'from_USA'}, inplace=True)

In [324]:
data['from_USA'].value_counts()

 United-States                 29170
 Mexico                          643
 ?                               583
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 

Видно, что граждане США составялют 89.5% всего датасета. Есть смысл привести столбец country к бинарному виду, где United-States = 1, остальные = 0.

In [325]:
data['from_USA'] = data['from_USA'].apply(lambda x: 1 if x == ' United-States' else 0)

In [326]:
data['from_USA'].value_counts()
# всё сходится

1    29170
0     3391
Name: from_USA, dtype: int64

столбец sex тоже переделаю в бинарный

In [327]:
data['sex'].value_counts()

 Male      21790
 Female    10771
Name: sex, dtype: int64

In [328]:
data['sex'] = data['sex'].map({' Male': 1, ' Female': 0})

In [293]:
data['sex'].value_counts()

1    21790
0    10771
Name: sex, dtype: int64

так же поступлю со столбцом race

In [329]:
data['race'] = data['race'].apply(lambda x: 1 if x == ' White' else 0)

In [295]:
data['race'].value_counts()

1    27816
0     4745
Name: race, dtype: int64

In [330]:
data.rename(columns={'race': 'white'}, inplace=True)

In [297]:
from sklearn.model_selection import train_test_split

In [332]:
X = pd.get_dummies(data.iloc[:, :-1])

In [333]:
X_train, X_test, y_train, y_test = train_test_split(X, data['income'], test_size=0.2, random_state=7)

### обучить любой классификатор (какой вам нравится)

In [299]:
import xgboost as xgb

In [342]:
y_train = y_train.astype(int)
y_test= y_test.astype(int)

In [343]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

[21:00:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [346]:
def evaluate_results(y_test, y_predict):
    """ Проверка качества модели с помощью DS-метрик"""
    from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict, average='binary')
    recall = recall_score(y_test, y_predict, average='binary')
    return f1, precision, recall, roc

In [347]:
evaluate_results(y_test, y_predict)

(0.7078534031413612, 0.7782041442824251, 0.649167733674776, 0.7953978438117365)

### далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [350]:
data2 = pd.get_dummies(data.copy())
data2['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [354]:
# отобрать P
pos_ind = np.where(data2.loc[:, 'income'].values == 1)[0]

# перемешать
np.random.shuffle(pos_ind)

# отставить 25% от Р
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Возьмем {pos_sample_len} из {len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Возьмем 1961 из 7841 as positives and unlabeling the rest


In [355]:
# Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
data2['class_test'] = -1
data2.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', data2.iloc[:, -1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


In [356]:
x_data = data2.drop(['income', 'class_test'], axis=1).values  # just the X
y_labeled = data2.loc[:, 'class_test'].values  # new class (just the P & U)
y_positive = data2.loc[:, 'income'].values  # original class

### применить random negative sampling для построения классификатора в новых условиях


In [367]:
data2 = data2.sample(frac=1)
neg_sample = data2[data2['class_test'] == -1][:len(data2[data2['class_test'] == 1])]
sample_test = data2[data2['class_test'] == -1][len(data2[data2['class_test'] == 1]):]
pos_sample = data2[data2['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(3921, 64) (3921, 64)


In [372]:
model_rns = xgb.XGBClassifier()
model.fit(sample_train.drop(['income', 'class_test'], axis=1).values,
          sample_train.loc[:, 'income'].values)
y_predict_rns = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:12:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.5397402125533655,
 0.3887725726249673,
 0.8823878823878823,
 0.8318130869414121)

### сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [374]:
pd.DataFrame([
    evaluate_results(y_test, y_predict),
    evaluate_results(sample_test.loc[:, 'income'].values, y_predict_rns)
], columns={'f1', 'precision', 'recall', 'roc'}, index={'Simple xgboost', 'random negative sampling'}).round(2)

,recall,f1,precision,roc
Simple xgboost,0.71,0.78,0.65,0.80
random negative sampling,0.54,0.39,0.88,0.83


### поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)